In [1]:
# -.-|m { input_fold: hide }
%load_ext pretty_jupyter

# <div align="center" style="padding: 10px; font-weight: bold;">Introduction</div>

In this assignment, we will look into the detection of melanoma using a Convolutional Neural Network (CNN) by applying transfer learning with a pre-trained model. I decided using the <a href="https://www.kaggle.com/datasets/bhaveshmittal/melanoma-cancer-dataset/" target="_blank"><b>🔬Melanoma Cancer Image Dataset</b></a>. It consists of 13 900 high-resolution images in test and train sets, both divided in two categorical fodlers - Benign and Malignant. The test folders comprise ot 1 000 photos each.

For the choice of the pre-trained model I searched for previous uses and researches. Based on the performance comparison in the <a href="https://www.biorxiv.org/content/10.1101/2020.05.22.110973v3" target="_blank"><b>Sagar (2020)</b></a> research I decided to opt for the <a href="https://keras.io/api/applications/resnet/#resnet50v2-function" target="_blank"><b>ResNet50V2</b></a> model.

In [3]:
a = 10
b = 5